<a href="https://colab.research.google.com/github/scarecrow021/Beam_Selector_Stress_Deformation/blob/main/Beam_Selector_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Input Form** 

In [186]:
caseNumber = 1 #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '-99'] {type:"raw"}
checkFor = "Stress" #@param ["Stress", "Deformation", "Debug"]
force =  150000#@param {type:"number"}
torque = 0 #@param {type:"number"}
lengthOfBeam =  17.4#@param {type:"number"}
length_a = 0 #@param {type:"number"}
length_b =  5#@param {type:"number"}
length_c = 0 #@param {type:"number"}
length_L = 0 #@param {type:"number"}
youngsModulus = 210 #@param {type:"number"}
stressLimit =  200#@param {type:"number"}
deformationLimit =  29#@param {type:"number"}


# Back End

### Library Import

In [187]:
from google.colab import files
import urllib.request
import os
import pandas as pd
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image, ImageFont, ImageDraw
import sys
import math
import matplotlib.pyplot as plt

# automatically uploads beamLibrary.xlsx from github repo (cannot work without this)
if os.path.isfile('/content/beamLibrary.xlsx'):
    print("File Exists! Carry on....")
else:
    url = 'https://github.com/scarecrow021/Beam_Selector_Stress_Deformation/blob/main/beamLibrary.xlsx?raw=true'
    urllib.request.urlretrieve(url, '/content/beamLibrary.xlsx')
    print("File Uploaded!!")

# automatically uploads arial.ttffrom github repo (font file for result printing)
if os.path.isfile('/content/arial.ttf'):
    print("File Exists! Carry on....")
else:
    url = 'https://github.com/scarecrow021/Beam_Selector_Stress_Deformation/blob/main/arial.ttf?raw=true'
    urllib.request.urlretrieve(url, '/content/arial.ttf')
    print("File Uploaded!!")


File Exists! Carry on....
File Exists! Carry on....


### Function Definitions


#### Deformation Limit Functions

In [188]:
## Deformation Limit Functions Definitions

def FindDeformation(Iy):
    if caseNumber == 1 :
        _deform = (1000*force*lengthOfBeam**3) / (48*Iy*youngsModulus*10**9)
    elif caseNumber == 2 :
        _deform = (1000*force*lengthOfBeam**3) / (3*Iy*youngsModulus*10**9)
    elif caseNumber == 3 :
        _deform = (1000*force*lengthOfBeam**3) / (192*Iy*youngsModulus*10**9)
    elif caseNumber == 4 :
        _deform = (1000*torque*lengthOfBeam**2) / (2*Iy*youngsModulus*10**9)
    elif caseNumber == 5 :
        _deform = (1000*torque*lengthOfBeam**2) / (8*Iy*youngsModulus*10**9)
    elif caseNumber == 6 :
        _deform = (1000*force*length_a*(3*lengthOfBeam**2 - 4*length_a**2)) / (24*Iy*youngsModulus*10**9)
    elif caseNumber == 7 :
        _deform = (1000*force*length_b*math.sqrt((lengthOfBeam**2 - length_b**2)**3)) / (9*math.sqrt(3)*lengthOfBeam*Iy*youngsModulus*10**9)
    elif caseNumber == 8 :
        _deform = (1000*force*(lengthOfBeam - length_b)**2 * (3*lengthOfBeam - (lengthOfBeam - length_b))) / (6*Iy*youngsModulus*10**9)
    elif caseNumber == 9 :
        _deform = (1000*2*force*length_b**3 * (lengthOfBeam-length_b)**2) / (3*Iy*(youngsModulus*10**9)*(lengthOfBeam + 2*length_b)**2)
    elif caseNumber == 10 :
        _deform = (1000*torque*lengthOfBeam**2) / (9*math.sqrt(3)*Iy*youngsModulus*10**9)
    elif caseNumber == 11 :
        _deform = (1000*force*lengthOfBeam**4) / (8*Iy*youngsModulus*10**9)
    elif caseNumber == 12 :
        _deform = (1000*5*force*lengthOfBeam**4) / (384*Iy*youngsModulus*10**9)
    elif caseNumber == 13 :
        _deform = (1000*force*length_a**2 * (length_L + length_a)) / (3*Iy*youngsModulus*10**9)
    elif caseNumber == 14 :
        _deform = (1000*force*length_a*length_b*length_c*(2*length_a + length_b)) / (6*(length_a + length_b)*Iy*youngsModulus*10**9)
    elif caseNumber == 15 :
        _deform = (1000*force*lengthOfBeam**4) / (384*Iy*youngsModulus*10**9)
    else:
        _deform = -1
    return _deform


def DeformLooper(beamLib, IyColNum, _result, _yes = False):
    for h in range(0,len(beamLib.index)):
        Iy = beamLib.iat[h,IyColNum]/(100**4)
        deform = FindDeformation(Iy)
        if deform < deformationLimit:
            if not _yes:
                _result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,1]) + " ...  Deformation: " + str(round(deform,2)) + " mm.")
            else:
                _result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,0]) + " ...  Deformation: " + str(round(deform,2)) + " mm.")
            break


def Case_Deform_Call():
    _result = []
    text1 = "You selected Case " + str(caseNumber) + " Deformation Limit"
    _result = [text1, "The minimum required beam and the deformation are: "] # Result String Array
    for libNum in range(0,15):
        beamLib = pd.read_excel('/content/beamLibrary.xlsx', libNum)
        if libNum in [8,13,14]:
            DeformLooper(beamLib, 5, _result, True)
        elif libNum == 12:
            DeformLooper(beamLib, 4, _result, True)   
        elif libNum in [7,9,10,11]:
            DeformLooper(beamLib, 3, _result, True)
        elif libNum in range(0,7):
            DeformLooper(beamLib, 6, _result)
    return _result


#### Stress Limit Functions

In [189]:
## Stress Limit Functions Definitions

def FindBendingMoment():
    if caseNumber == 1:
        M = force*lengthOfBeam/4
    elif caseNumber == 2:
        M = force*lengthOfBeam
    elif caseNumber == 3:
        M = force*lengthOfBeam/8
    elif caseNumber == 4:
        M = torque
    elif caseNumber == 5:
        M = torque
    elif caseNumber == 6:
        M = force*length_a
    elif caseNumber == 7:
        M = force*(lengthOfBeam - length_b)*length_b/lengthOfBeam
    elif caseNumber == 8:
        M = force*(lengthOfBeam - length_b)
    elif caseNumber == 9:
        M = force*(lengthOfBeam - length_b)*length_b/(lengthOfBeam**2)
    elif caseNumber == 10:
        M = torque
    elif caseNumber == 11:
        M = (force*lengthOfBeam**2)/2
    elif caseNumber == 12:
        M = (force*lengthOfBeam**2)/8
    elif caseNumber == 13:
        M = force*length_a
    elif caseNumber == 14:
        M = force*length_a*length_b/(length_a + length_b)
    elif caseNumber == 15:
        M = (force*lengthOfBeam**2)/12
    return M


def FindStress(M, h, I):                         # Be careful ymax is not h/2 here as in Excel. That is why it is divided by 2 in the formula.
    _stress = (M*(h/(2*1000)))/((I*10**-8)*10**6)
    return _stress


def StressLimitCheck(M, stressLimit, result, beamLib, Iy_columNum, _yes= False):
    hDim_columNum=1
    for h in range(0,len(beamLib.index)):
        Stress = FindStress(M, beamLib.iat[h,hDim_columNum], beamLib.iat[h,Iy_columNum]) 
        if Stress < stressLimit:
            if not _yes:
                result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,1]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
            else:
                result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,0]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
            break


def StressLimitCheckLooper(result, stressLimit):
    M = FindBendingMoment()
    stressResArray = []
    for libNum in range(0,15):
        beamLib = pd.read_excel('/content/beamLibrary.xlsx', libNum)
        if libNum in [8,13,14]:
            StressLimitCheck(M, stressLimit, result, beamLib, 5, True)
        elif libNum == 12:
            StressLimitCheck(M, stressLimit, result, beamLib, 4, True)
        elif libNum in [7,9,10,11]:
            StressLimitCheck(M, stressLimit, result, beamLib, 3, True)
        elif libNum in range(0,7):
            StressLimitCheck(M, stressLimit, result, beamLib, 6, False)
    return result




def Case_Stress_Call():
    _result = []
    text1 = "You selected Case " + str(caseNumber) + " Stress Limit"
    _result = [text1, "The minimum required beam and the stress are: "] # Result String Array
    _result = StressLimitCheckLooper(_result, stressLimit)
    
    return _result

In [190]:
def StressArrayFiller(M, stressLimit, beamLib, Iy_columNum, _yes= False):
    stressArr = np.empty(464)
    hDim_columNum=1
    for h in range(0,len(beamLib.index)):
        Stress = FindStress(M, beamLib.iat[h,hDim_columNum], beamLib.iat[h,Iy_columNum])
        stressArr[h] = Stress
    return stressArr

def FindMax():
    _max = 0
    for libNum in range(0,15):
        beamLib = pd.read_excel('/content/beamLibrary.xlsx', libNum)
        if libNum in [8,13,14]:
            a =  len(beamLib.index)
            _max = max(a,_max)
        elif libNum == 12:
            a =  len(beamLib.index)
            _max = max(a,_max)
        elif libNum in [7,9,10,11]:
            a =  len(beamLib.index)
            _max = max(a,_max)
        elif libNum in range(0,7):
            a =  len(beamLib.index)
            _max = max(a,_max)
    print(_max)

def PlotStress():
    M = FindBendingMoment()
    stressResMat = np.zeros((1,464))
    for libNum in range(0,15):
        beamLib = pd.read_excel('/content/beamLibrary.xlsx', libNum)
        if libNum in [8,13,14]:
            stressRow = StressArrayFiller(M, stressLimit, beamLib, 5, True)
            stressResMat = np.r_[stressResMat,[stressRow]] 
        elif libNum == 12:
            stressRow = StressArrayFiller(M, stressLimit, beamLib, 4, True)
            stressResMat = np.r_[stressResMat,[stressRow]]
        elif libNum in [7,9,10,11]:
            stressRow = StressArrayFiller(M, stressLimit, beamLib, 3, True)
            stressResMat = np.r_[stressResMat,[stressRow]]
        elif libNum in range(0,7):
            stressRow = StressArrayFiller(M, stressLimit, beamLib, 6, False)
            stressResMat = np.r_[stressResMat,[stressRow]]
    return stressResMat

## Interface

In [191]:
def CalculatorCaller():
    if caseNumber in range(1,16):
        if checkFor == "Stress":
            _result = Case_Stress_Call()
        elif checkFor == "Deformation":
            _result = Case_Deform_Call()
        else:
            _result = "INVALID"
    else:
        _result = "INVALID"
    return _result

In [192]:
# ## Calls Calculator
# result = CalculatorCaller()
# ## Result Beautification
# # Empty White Picture Creation
# whiteFrame = np.full((500, 500, 3), 255, dtype = np.uint8)
# cv2.imwrite('/content/result.jpeg', whiteFrame)
# # Result text insertion
# title_text = "*************** RESULT *************** \n"
# resultImage = Image.open("/content/result.jpeg")
# for i in range(0,len(result)):
#     title_text = title_text + result[i] + "\n"
# font = ImageFont.truetype("/content/arial.ttf", 16)
# image_editable = ImageDraw.Draw(resultImage)
# image_editable.text((10,10), title_text, fill=(0, 0, 0), font = font)
# resultImage.save("/content/result.jpeg")
# # Re-read image with opencv
# image = cv2.imread("/content/result.jpeg")
stressMat = PlotStress()

In [ ]:
stressMat = np.delete(stressMat, 0, 0)
plt.matshow(stressMat);
plt.colorbar()
plt.show()

# **RESULT**

In [ ]:
cv2_imshow(image)